In [1]:
import pandas as pd
from neo4j import GraphDatabase
from tqdm import tqdm

import os
import sys
eval_dir = os.path.join(os.path.realpath('..'), 'akbc_2021/eval')
sys.path.append(eval_dir) 
import utils as ut

In [2]:
# setup database connection
driver = GraphDatabase.driver('bolt://localhost:7687', auth=('neo4j', 'fb14'))
sess = driver.session()

In [3]:
for include_sibling in [True, False]:
    print('include_sibling: ', include_sibling)
    
    if not include_sibling:
        sibling_clause = "WHERE type(r) <> 'sibling'"
        out_dir = '../data/fb13/resplit/'
    else:
        sibling_clause = ""
        out_dir = '../data/fb13/resplit_with_sibs/'
    
    q_out = f'''
    MATCH (n)-[r]->(x)
    {sibling_clause}
    RETURN n.name as entity, count(distinct r) as out_degree
    '''

    q_in = f'''
    MATCH (n)<-[r]-(x)
    {sibling_clause}
    RETURN n.name as entity, count(distinct r) as in_degree
    '''

    q_total = f'''
    MATCH (n)-[r]-(x)
    {sibling_clause}
    RETURN n.name as entity, count(distinct r) as total_degree
    '''

    q_all = '''
    MATCH (n)
    RETURN n.name as entity, labels(n) as type
    '''
    
    all_nodes = ut.res_to_df(sess.run(q_all))
    all_nodes = all_nodes.explode(column='type') # reformat singleton label lists

    in_degree = ut.res_to_df(sess.run(q_in))
    out_degree = ut.res_to_df(sess.run(q_out))
    total_degree = ut.res_to_df(sess.run(q_total)) # I confirmed that this is equivalent to summing in_degre and out_degree

    degree = all_nodes.merge(in_degree, on='entity', how='left')
    degree = degree.merge(out_degree, on='entity', how='left')
    degree = degree.merge(total_degree, on='entity', how='left')
    degree.fillna(0, inplace=True)

    degree.shape
    degree.head()
    
    degree.to_csv(f'{out_dir}/node_degree.tsv', sep='\t', index=False)

include_sibling:  True


(74845, 5)

,entity,type,in_degree,out_degree,total_degree
0,charles_studd,Person,0.0,5.0,5
1,villach,Location,5.0,0.0,5
2,vesta_m_roy,Person,0.0,4.0,4
3,wirt_c_rowland,Person,0.0,4.0,4
4,metairie,Location,3.0,0.0,3


include_sibling:  False


(74845, 5)

,entity,type,in_degree,out_degree,total_degree
0,charles_studd,Person,0.0,5.0,5
1,villach,Location,5.0,0.0,5
2,vesta_m_roy,Person,0.0,4.0,4
3,wirt_c_rowland,Person,0.0,4.0,4
4,metairie,Location,3.0,0.0,3
